In [2]:
# Import the library and APIs
import pandas as pd
import seaborn as sns
import matplotlib as mp1
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import csv
import array as arr
import tensorflow as tf
import random
import timeit

from pandas import DataFrame
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from numba import cuda

In [3]:
# Crate a object class call Person
# Which is used to store the value
class Person:
    def __init__(self, id, filename, img, gender, ageGroup, age):
        self.id = id
        self.filename = filename
        self.img = img
        self.gender = gender
        self.ageGroup = ageGroup
        self.age = age
        self.guessGender = 0
        self.guessAgeGroup = 0
        self.guessAge = 0


In [4]:
# Read the .csv file to get the subjects informations
# Read the .txt file to get the training id list
%matplotlib inline

df=pd.read_csv('Age_gender_info.csv')
id_train = np.loadtxt("GEI_IDList_train.txt", dtype='int32')

In [ ]:
# The total number of epochs should over 100
# After 100, the model will be saved 
# only when the test ass higher than acc

def training(x_train,y_train,x_test,y_test,model,epochs,acc,address,isFirst):
    e = 0
    e_a = 0
    if isFirst:
        e+=100
        history = model.fit(x_train, y_train, epochs = e)
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 1)
        acc = test_acc
        e_a = e
        model.save(address)
    for e < epochs:
        e+=5
        history = model.fit(x_train, y_train, epochs = e)
        test_loss, test_acc = model.evaluate(x_test, y_test, verbose = 1)
        if (test_acc > acc):
            acc = test_acc
            e_a = e
            model.save(address)
    return e_a, acc

In [5]:
df

,ID,Age,Gender
0,16588,43,F
1,44531,32,M
2,42842,20,F
3,91844,21,M
4,39976,11,M
...,...,...,...
63841,55628,66,F
63842,37834,9,F
63843,90955,46,M
63844,89684,28,M


In [10]:
datadir = r"C:/Users/Acer/FYP/GEI/"

id = df['ID'][0].astype(str)
filename = os.path.join(datadir, '0'*(8-len(id)) + id + '.png')
img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

height, width = img.shape

In [6]:
# Data mapping
# Loop through df, get the data on each rows
# Create a new Person object and set the value
# Append the object to a object list

datadir = r"C:/Users/Martin/FYP/GEI/"
people_train = []
people_test = []

df.ID = df.ID.astype(str)
for index, data in df.iterrows():
    id = data.ID
    filename = os.path.join(datadir, '0'*(8-len(id)) + id + '.png')
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

    gender = 1 if data.Gender == 'M' else 0

    age = data.Age
    if age < 6:
        ageGroup = 0
    elif age < 11:
        ageGroup = 1
    elif age < 16:
        ageGroup = 2
    elif age < 61:
        ageGroup = 3
    else:
        ageGroup = 4

    p = Person(id, filename, img, gender, ageGroup, age)

    if int(id) in id_train:
        people_train.append(p)
    else:
        people_test.append(p)
        
all_people = people_train + people_test

# Getting the height and width of a image
# Since all the images have the same value of height and width
height, width = all_people[0].img.shape

# Print out the length of all_people list
# It should be 63846
print(len(all_people))

63846


In [7]:
all_people = people_train + people_test

# Gender

In [10]:
x_train = []
y_train = []

for p in people_train:
    img = np.rot90(p.img,3)
    x_train.append(img)
    y_train.append(p.gender)
    
x_train = np.array(x_train).reshape(-1, width, height, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

In [11]:
x_test = []
y_test = []

for p in people_test:
    img = np.rot90(p.img,3)
    x_test.append(img)
    y_test.append(p.gender)
    
x_test = np.array(x_test).reshape(-1, width, height, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

In [34]:
model = Sequential()
model.add(Conv2D(81, (5, 5), strides = 1, activation = 'relu', input_shape = (width, height, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (3,3), strides = 2))

model.add(Conv2D(45, (7, 7), strides = 1, activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2), strides = 2))

model.add(Flatten())
model.add(Dense(1024, activation = 'relu'))
model.add(BatchNormalization())

model.save("OULP-Age_without_output_layer.h5")

In [50]:
# Create the block for layer 1
block_gender = tf.keras.models.load_model('OULP-Age_without_output_layer.h5')

block_gender.add(Dense(2, activation = 'softmax'))
block_gender.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_gender.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 84, 124, 81)       2106      
_________________________________________________________________
batch_normalization_9 (Batch (None, 84, 124, 81)       324       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 41, 61, 81)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 35, 55, 45)        178650    
_________________________________________________________________
batch_normalization_10 (Batc (None, 35, 55, 45)        180       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 27, 45)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20655)            

In [51]:
history = block_gender.fit(x_train, y_train, epochs = 100)
test_loss, test_acc = block_gender.evaluate(x_test, y_test, verbose = 1)
print(test_acc)

Epoch 1/100
998/998 [==============================] - 1480s 1s/step - loss: 0.1889 - accuracy: 0.9319
Epoch 2/100
998/998 [==============================] - 1512s 2s/step - loss: 0.1340 - accuracy: 0.9488
Epoch 3/100
998/998 [==============================] - 1542s 2s/step - loss: 0.1248 - accuracy: 0.9521
Epoch 4/100
998/998 [==============================] - 1622s 2s/step - loss: 0.1136 - accuracy: 0.9580
Epoch 5/100
998/998 [==============================] - 1702s 2s/step - loss: 0.1087 - accuracy: 0.9594
Epoch 6/100
998/998 [==============================] - 1604s 2s/step - loss: 0.0990 - accuracy: 0.9629
Epoch 7/100
998/998 [==============================] - 1735s 2s/step - loss: 0.0930 - accuracy: 0.9652
Epoch 8/100
998/998 [==============================] - 1756s 2s/step - loss: 0.0858 - accuracy: 0.9678
Epoch 9/100
998/998 [==============================] - 1815s 2s/step - loss: 0.0808 - accuracy: 0.9696
Epoch 10/100
998/998 [==============================] - 1725s 2s/step - l

In [52]:
#0.9339348077774048
block_gender.save("OULP-Age_gender.h5")

In [43]:
#model.save("gender_model_v2.h5")

In [14]:
block_gender = tf.keras.models.load_model('OULP-Age_gender.h5')

In [9]:
gender_label = ['F','M']

start = timeit.default_timer()
for p in all_people:
    img = np.rot90(p.img,3)
    img = np.array(img).reshape(-1, width, height, 1)
    prediction = block_gender.predict(img)
    p.guessGender = np.argmax(prediction)

    b = "ID: "+ p.id + " Gender: " + gender_label[p.gender] + " Guess: " + gender_label[p.guessGender]
    print (b, end="\r")
        
stop = timeit.default_timer()

In [12]:
print('Prediction Time: ', round((stop - start)/60), 'minutes\n')

Prediction Time:  42 minutes



# Female age group

In [23]:
x_train = []
y_train = []

for p in people_train:
    if p.guessGender == 0:
        img = np.rot90(p.img,3)
        x_train.append(img)
        y_train.append(p.ageGroup)
    
x_train = np.array(x_train).reshape(-1, width, height, 1)
y_train = np.array(y_train, dtype='int32')

y_train = to_categorical(y_train)

x_test = []
y_test = []

for p in people_test:
    if p.guessGender == 0:
        img = np.rot90(p.img,3)
        x_test.append(img)
        y_test.append(p.ageGroup)
    
x_test = np.array(x_test).reshape(-1, width, height, 1)
y_test = np.array(y_test, dtype='int32')

y_test = to_categorical(y_test)

In [24]:
# Create the block for layer 2 female
block_F_age_group = tf.keras.models.load_model('OULP-Age_without_output_layer.h5')

block_F_age_group.add(Dense(5, activation = 'softmax'))
block_F_age_group.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
block_F_age_group.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 84, 124, 81)       2106      
_________________________________________________________________
batch_normalization_9 (Batch (None, 84, 124, 81)       324       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 41, 61, 81)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 35, 55, 45)        178650    
_________________________________________________________________
batch_normalization_10 (Batc (None, 35, 55, 45)        180       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 27, 45)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 20655)            

In [ ]:
start = timeit.default_timer()

history = block_F_age_group.fit(x_train, y_train, epochs = 100)
test_loss, test_acc = block_F_age_group.evaluate(x_test, y_test, verbose = 1)
print(test_acc)

stop = timeit.default_timer()
print('Training Time: ', round((stop - start)/60), 'minutes\n')

In [ ]:
block_F_age_group.save("OULP-Age_F_age_group.h5")

In [ ]:
def to_cat(y_arr,min,max):
    y = []
    for age in range(y_arr):
        arr_range = [0]*(max - min + 1)
        arr_range[age-min]+=1
        y.append(arr_range)
    return np.asarray(y)

In [ ]:
def un_cat(prediction, group):
    min_list = [0, 6, 11, 16, 61]
    return min_list[group]+np.argmax(prediction)
    

# ==============================================

In [44]:
gender_model = tf.keras.models.load_model('gender_model_v2.h5')

In [55]:
error_count = []

start = timeit.default_timer()
for p in people_test:
    img = np.array(p.img).reshape(-1, IMG_SIZE,IMG_SIZE,1)
    if p.guessGender == 0:
        prediction = gender_model.predict(img)
    else:
        prediction = gender_model.predict(img)
    p.guessAgeGroup = np.argmax(prediction)
    
stop = timeit.default_timer()


In [56]:
print('Prediction Time: ', round((stop - start)/60), 'minutes\n')

Prediction Time:  14 minutes



In [145]:
error_female = []
error_male = []
for p in error_count:
    if p.gender == 0:
        error_female.append(p)
    else:
        error_male.append(p)
        
print(len(error_female))
print(len(error_male))

786
221


In [146]:
pople_test_female_count = 0
for p in people_test:
    if p.gender == 0:
        pople_test_female_count += 1



In [147]:
pople_test_male_count = len(people_test) - pople_test_female_count
true_male = pople_test_male_count - len(error_male)
true_female = pople_test_female_count - len(error_female)
guess_to_male = true_male + len(error_female)
guess_to_female = true_female + len(error_male)
total = guess_to_male + guess_to_female

In [148]:
dframe = DataFrame([[true_male,len(error_female),guess_to_male],
                    [len(error_male),true_female,guess_to_female],
                    [pople_test_male_count,pople_test_female_count,total]],
                   index = ['male', 'female', 'Total'], columns = ['male', 'female', 'Total'])

dframe.index.names = ['Predicted']
dframe.columns.names = ['       Ground Truth-->']
print(dframe)

       Ground Truth-->   male  female  Total
Predicted                                   
male                    15276     786  16062
female                    221   15640  15861
Total                   15497   16426  31923
